In [1]:
!pip install mljar-supervised

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.5/91.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 2.2 MB/s eta 0:00:00
  Created wheel for mljar-supervised: filename=mljar_supervised-0.11.5-py3-none-any.whl size=147060 sha256=f472f19f05ab52fac2e313c6c82e9ef575d9bdbb254b1892a2852e901e9985e6
  Stored in directory: /root/.cache/pip/whee

In [2]:
import pandas as pd
import numpy as np
import random
import os
from supervised.automl import AutoML
import pandas as pd

import numpy as np
import random
import os
import gc
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(41) # 행운의 seed

In [4]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

In [4]:
train.corr()

,Exercise_Duration,Body_Temperature(F),BPM,Height(Feet),Height(Remainder_Inches),Weight(lb),Age,Calories_Burned
Exercise_Duration,1.000000,0.904314,0.853489,-0.004566,0.003649,0.002236,0.017734,0.954783
Body_Temperature(F),0.904314,1.000000,0.770707,0.004248,-0.003311,0.007554,0.016474,0.824379
BPM,0.853489,0.770707,1.000000,0.003110,0.004244,0.012562,0.014702,0.899900
Height(Feet),-0.004566,0.004248,0.003110,1.000000,-0.536675,0.821749,0.015356,0.020780
Height(Remainder_Inches),0.003649,-0.003311,0.004244,-0.536675,1.000000,-0.025702,-0.011303,-0.004381
Weight(lb),0.002236,0.007554,0.012562,0.821749,-0.025702,1.000000,0.090536,0.042568
Age,0.017734,0.016474,0.014702,0.015356,-0.011303,0.090536,1.000000,0.159624
Calories_Burned,0.954783,0.824379,0.899900,0.020780,-0.004381,0.042568,0.159624,1.000000


In [5]:
train["Height"] = train.pop("Height(Feet)") * 30.48 + train.pop("Height(Remainder_Inches)") * 2.54
test["Height"] = test.pop("Height(Feet)") * 30.48 + test.pop("Height(Remainder_Inches)") * 2.54

train["Weight"] = train.pop("Weight(lb)") * 0.453592
test["Weight"] = test.pop("Weight(lb)") * 0.453592

train["Body_Temperature"] = (train.pop("Body_Temperature(F)") - 32) / 1.8
test["Body_Temperature"] = (test.pop("Body_Temperature(F)") - 32) / 1.8

In [7]:
train

,ID,Exercise_Duration,BPM,Weight_Status,Gender,Age,Calories_Burned,Height,Weight,Body_Temperature
0,TRAIN_0000,26.0,107.0,Normal Weight,F,45,166.0,175.26,69.989246,40.888889
1,TRAIN_0001,7.0,88.0,Overweight,M,50,33.0,198.12,102.012841,39.611111
2,TRAIN_0002,7.0,86.0,Overweight,M,29,23.0,190.50,99.019134,39.611111
3,TRAIN_0003,17.0,99.0,Normal Weight,F,33,91.0,167.64,66.995538,40.000000
4,TRAIN_0004,9.0,88.0,Normal Weight,M,38,32.0,177.80,77.019922,39.277778
...,...,...,...,...,...,...,...,...,...,...
7495,TRAIN_7495,22.0,104.0,Normal Weight,F,75,151.0,147.32,50.983741,40.611111
7496,TRAIN_7496,20.0,104.0,Normal Weight,F,21,114.0,172.72,66.995538,40.722222
7497,TRAIN_7497,8.0,90.0,Overweight,M,57,41.0,187.96,91.988458,39.500000
7498,TRAIN_7498,12.0,97.0,Overweight,M,35,57.0,175.26,76.022019,40.222222


In [8]:
def calculate_intensity(bpm):
    """
    운동 강도 계산 함수
    bpm: 분당 심박수
    """
    if bpm < 100:
        return 0
    elif bpm < 130:
        return 1
    else:
        return 2

# 운동 강도 계산 후 새로운 열로 추가
train['Intensity'] = train['BPM'].apply(calculate_intensity)
test['Intensity'] = test['BPM'].apply(calculate_intensity)

In [9]:
train.head()

,ID,Exercise_Duration,BPM,Weight_Status,Gender,Age,Calories_Burned,Height,Weight,Body_Temperature,Intensity
0,TRAIN_0000,26.0,107.0,Normal Weight,F,45,166.0,175.26,69.989246,40.888889,1
1,TRAIN_0001,7.0,88.0,Overweight,M,50,33.0,198.12,102.012841,39.611111,0
2,TRAIN_0002,7.0,86.0,Overweight,M,29,23.0,190.50,99.019134,39.611111,0
3,TRAIN_0003,17.0,99.0,Normal Weight,F,33,91.0,167.64,66.995538,40.000000,0
4,TRAIN_0004,9.0,88.0,Normal Weight,M,38,32.0,177.80,77.019922,39.277778,0


In [6]:
train["BMI"] = train["Weight"] / (train["Height"] / 100)**2
test["BMI"] = test["Weight"] / (test["Height"] / 100)**2

train = train.drop("Weight_Status", axis=1)
test = test.drop("Weight_Status", axis=1)

train["Gender"] = train["Gender"].map({"F": 0, "M": 1})
test["Gender"] = test["Gender"].map({"F": 0, "M": 1})

train["Calories_Burned"] = np.log1p(train["Calories_Burned"])

train["Age"] = np.log(train["Age"])
test["Age"] = np.log(test["Age"])

body_temp_mean = train["Body_Temperature"].mean()
body_temp_std = train["Body_Temperature"].std()
train["Body_Temperature"] = (((train["Body_Temperature"] - body_temp_mean) / body_temp_std) + 4)**3
test["Body_Temperature"] = (((test["Body_Temperature"] - body_temp_mean) / body_temp_std) + 4)**3

In [7]:
numeric_columns = train.columns[train.dtypes == np.float]
target_column = "Calories_Burned"

In [12]:
# target
minmax_scaler = StandardScaler()  
train[target_column] = minmax_scaler.fit_transform(train[[target_column]])

# target외의 양적 변수
tmp_scaler = StandardScaler()
train[numeric_columns.drop(target_column)] = tmp_scaler.fit_transform(train[numeric_columns.drop(target_column)])
test[numeric_columns.drop(target_column)] = tmp_scaler.transform(test[numeric_columns.drop(target_column)])

In [13]:
train

,ID,Exercise_Duration,BPM,Gender,Age,Calories_Burned,Height,Weight,Body_Temperature,Intensity,BMI
0,TRAIN_0000,1.256628,1.199774,0,0.332281,1.001706,0.052654,-0.332109,1.399022,1,-0.990606
1,TRAIN_0001,-1.017530,-0.782140,1,0.595023,-0.653751,1.649312,1.787635,-0.788351,0,1.046098
2,TRAIN_0002,-1.017530,-0.990762,1,-0.763385,-1.016025,1.117092,1.589472,-0.788351,0,1.869905
3,TRAIN_0003,0.179395,0.365284,0,-0.441165,0.381593,-0.479565,-0.530272,-0.291806,0,-0.320996
4,TRAIN_0004,-0.778145,-0.782140,1,-0.089351,-0.684801,0.230061,0.133274,-1.115388,0,0.012359
...,...,...,...,...,...,...,...,...,...,...,...
7495,TRAIN_7495,0.777858,0.886840,0,1.606148,0.903819,-1.898816,-1.590144,0.779137,1,-0.542092
7496,TRAIN_7496,0.538473,0.886840,0,-1.568298,0.613684,-0.124752,-0.530272,1.016446,1,-1.199395
7497,TRAIN_7497,-0.897837,-0.573517,1,0.921774,-0.433969,0.939686,1.124089,-0.906861,0,1.076724
7498,TRAIN_7498,-0.419067,0.156662,1,-0.294432,-0.098252,0.052654,0.067220,0.053904,0,0.258028


In [8]:
# 독립변수로 설정할 train_x에서는 종속변수를 제거합니다. 또한 분석에 활용하지 않는 ID 데이터를 제거합니다.
train_x = train.drop(['ID', 'Calories_Burned'], axis = 1)
# train_y 변수를 종속변수로 사용하기 위해 Calories_Burned 데이터를 지정하였습니다.
train_y = train['Calories_Burned']

# train_x 데이터와 마찬가지로 분석에 활용하지 않는 ID 데이터를 제거합니다.
test_x = test.drop('ID', axis = 1)

In [9]:
test_x

,Exercise_Duration,BPM,Gender,Age,Height,Weight,Body_Temperature,BMI
0,26.0,107.0,0,3.806662,154.94,51.981643,107.497644,21.653241
1,29.0,111.0,1,3.044522,187.96,89.992653,98.179511,25.472770
2,11.0,90.0,1,4.060443,175.26,77.019922,62.866142,25.074799
3,24.0,108.0,1,3.555348,182.88,88.994750,122.552068,26.609209
4,29.0,116.0,0,4.204693,170.18,66.995538,156.749543,23.132834
...,...,...,...,...,...,...,...,...
7495,28.0,106.0,1,3.931826,182.88,81.011531,133.331320,24.222246
7496,4.0,84.0,1,3.218876,177.80,83.007336,16.790070,26.257475
7497,24.0,97.0,0,3.737670,157.48,59.012319,107.497644,23.795338
7498,8.0,83.0,0,3.871201,165.10,61.008124,37.233757,22.381723


In [10]:
automl = AutoML(mode="Compete", eval_metric='rmse',total_time_limit=3600*2)

In [ ]:
automl.fit(train_x, train_y)

AutoML directory: AutoML_1
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Decision Tree', 'Linear', 'Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Neural Network', 'Nearest Neighbors']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree rmse 0.228774 trained in 1.32 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 10-fold CV Shuffle
* Step simple_algorithms will try to check up to 4 models
1_DecisionTree rmse 0.239187 trained in 3.91 seconds
2_DecisionTree rmse 0.198141 trained in 2.68 seconds
3_DecisionTree rmse 0.198141 trained in 3.39 seconds
4_Linear 

In [11]:
preds = automl.predict(test_x)

In [12]:
original_predictions = np.expm1(preds)


In [13]:
original_predictions

array([172.91414245, 189.659026  ,  53.38801413, ..., 130.20022219,
        32.18654902, 191.26626596])

In [ ]:
original_y_pred = minmax_scaler.inverse_transform(preds.reshape(-1, 1))

In [ ]:
original_y_pred 

In [ ]:
original_predictions = np.expm1(original_y_pred)

In [ ]:
original_predictions 

In [ ]:
original_predictions

In [ ]:
import gc
gc.collect()

In [14]:
submission = pd.read_csv('/content/sample_submission.csv')

In [15]:
submission['Calories_Burned'] = original_predictions

In [16]:
submission.to_csv('/content/666.csv', index = False)